In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

#!pip install quilt
#!quilt install ResidentMario/missingno_data
from quilt.data.ResidentMario import missingno_data

%matplotlib inline

Set the option as 1 to select the products with most volatile reviews:

In [54]:
debug=True
option=2

## Read Datasets

In [3]:
month = ["March", "April", "May", "June", "July", "August", "September", "October", "November"]
avery_sales_rank = pd.DataFrame()
for i in range(0,len(month)):
    sr_file_name = "SalesRankExport_Avery_" + month[i] + "2019.csv"
    df_bin = pd.read_csv(sr_file_name)
    avery_sales_rank = pd.concat([avery_sales_rank, df_bin], ignore_index = True)

/Users/yvettepeng/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (17,18,20,21,23,24,26,27,29,30,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/yvettepeng/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/yvettepeng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [4]:
avery_sales_rank.columns

Index(['category_id1', 'category_id10', 'category_id2', 'category_id3',
       'category_id4', 'category_id5', 'category_id6', 'category_id7',
       'category_id8', 'category_id9', 'category_name1', 'category_name10',
       'category_name2', 'category_name3', 'category_name4', 'category_name5',
       'category_name6', 'category_name7', 'category_name8', 'category_name9',
       'category_rank1', 'category_rank10', 'category_rank2', 'category_rank3',
       'category_rank4', 'category_rank5', 'category_rank6', 'category_rank7',
       'category_rank8', 'category_rank9', 'date', 'end_ts', 'id', 'source',
       'start_ts'],
      dtype='object')

In [5]:
avery_review = pd.read_csv('Avery_reviews.csv')
avery_price = pd.read_csv('Avery_price.csv')

/Users/yvettepeng/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (2,3,8,11,12,24,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/yvettepeng/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1,20,21,22,23,28,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
avery_review.columns

Index(['source', 'product', '100 PL_custom', 'BU_custom', 'Benchmark_custom',
       'Brand_custom', 'Business Unit_custom', 'Category_custom',
       'License_custom', 'MPN_custom', 'Product Title_custom', 'SMU_custom',
       'Sprint_custom', 'Subcategory_custom', 'UPC_custom',
       'WM_Tracker_custom', 'name', 'date', 'status', 'sentiment', 'topic',
       'notes', 'author', 'verified', 'vine', 'stars', 'pvotes', 'tvotes',
       'title', 'text', 'image', 'video', 'reviewid', 'reviewlink', 'parent',
       'statusdt', 'statustime', 'helpfulcount', 'commenttext',
       'commentauthor', 'officialcomment', 'totalcomments', 'commentts',
       'commentdatestring', 'inputtime'],
      dtype='object')

In [14]:
avery_sales_rank['source'].unique()

array(['amazon'], dtype=object)

In [9]:
avery_review['source'].unique()

array(['amazon'], dtype=object)

## Select Products

In [18]:
avery_sales_rank.groupby(by='category_id1', as_index=False).agg({'id': pd.Series.nunique}).sort_values(by = 'id', ascending=False)

,category_id1,id
65,office-products/1069410,371
92,office-products/1069636,338
147,office-products/490930011,278
82,office-products/1069588,244
60,office-products/1069392,190
...,...,...
120,office-products/490549011,1
36,electronics/760796,1
124,office-products/490567011,1
86,office-products/1069602,1


In [19]:
# Select 10 option 1 products
products_volatile = avery_sales_rank[avery_sales_rank['category_id1'] == 'office-products/1069410'][['id', 'category_rank1']].drop_duplicates().groupby('id').std().sort_values(by = 'category_rank1', ascending = False).head(10)

In [31]:
products_volatile = products_volatile.reset_index()["id"]

In [32]:
products_volatile

0    B01FD4YTGS
1    B077K3LB79
2    B00XW395WQ
3    B01F291CXW
4    B00UHZM7JK
5    B079T4V9Q7
6    B00WH6LUU4
7    B07BLGZB3Q
8    B010DS4WWA
9    B0160P8JGK
Name: id, dtype: object

In [21]:
## select 10 option 2 products
products_most_review = avery_review[avery_review['product'].isin(avery_sales_rank[avery_sales_rank['category_id1'] \
                    == 'office-products/1069410']['id'].unique())][['product','reviewid']].groupby('product').nunique().sort_values(by = 'reviewid', ascending = False).head(10)


In [23]:
products_most_review = products_most_review.drop(['product'], axis=1).reset_index()["product"]

In [58]:
products_most_review

0    B001PF1BSE
1    B002JGQGQ0
2    B000ANLD1M
3    B0069RY9BY
4    B005IMAKYG
5    B00004Z5QO
6    B00004Z5SB
7    B01DJBMLV2
8    B000093L1J
9    B005ZXCWFI
Name: product, dtype: object

In [55]:
if(option==1):
    # more reviews and highly variant sales rank
    common = products_volatile
elif(option==2):
    # products with most reviews
    common = products_most_review
    pass

## Filter the datasets based on products

In [56]:
if(debug):
    avery_sales_rank_filtered = avery_sales_rank[avery_sales_rank['id'].isin(common)].set_index('id',drop=True)
    #asr1 = asr1.set_index('id',drop = True)
    avery_review_filtered = avery_review[avery_review['product'].isin(common)].set_index('product',drop=True)
    avery_price_filtered = avery_price[avery_price['id'].isin(common)].set_index('id',drop=True)
else:
    avery_sales_rank_filtered = avery_sales_rank.set_index('id',drop = True)
    # Same as above, create another dictionary for review data
    avery_review_filtered = avery_review.set_index('product',drop = True)
    avery_price_filtered = avery_price.set_index('id',drop=True)

In [57]:
avery_sales_rank_filtered.head()

,category_id1,category_id10,category_id2,category_id3,category_id4,category_id5,category_id6,category_id7,category_id8,category_id9,...,category_rank4,category_rank5,category_rank6,category_rank7,category_rank8,category_rank9,date,end_ts,source,start_ts
id,,,,,,,,,,,,,,,,,,,,,
B00004Z5QO,office-products/1069410,NaN,office-products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-11 19:34:59,2019-03-31,amazon,2019-03-01
B00004Z5QO,office-products/1069410,NaN,office-products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-12 19:49:09,2019-03-31,amazon,2019-03-01
B00004Z5QO,office-products/1069410,NaN,office-products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-13 20:08:07,2019-03-31,amazon,2019-03-01
B00004Z5QO,office-products/1069410,NaN,office-products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-14 20:52:36,2019-03-31,amazon,2019-03-01
B00004Z5QO,office-products/1069410,NaN,office-products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-15 19:37:16,2019-03-31,amazon,2019-03-01


## Create Dictionaries

In [59]:
# create sales rank dictionary
avery_sales_rank_dict = {}
for i in avery_sales_rank_filtered.index.unique():
    avery_sales_rank_dict[i] = avery_sales_rank_filtered.loc[i, ['source','date', 'category_id1',
       'category_name1', 'category_rank1', 'category_id2', 'category_name2', 'category_rank2']]

In [60]:
item_list=list(avery_sales_rank_dict.keys())
print(len(item_list))

10


In [61]:
# create review dictionary
avery_review_dict = {}
for i in avery_review_filtered.index.unique():
    avery_review_dict[i] = avery_review_filtered.loc[i, ['source','date','author', 'verified', 'vine', 'stars', 'pvotes', 'tvotes',
       'title', 'text', 'image', 'video']]

In [62]:
# create price dictionary
avery_price_dict = {}
for i in avery_price_filtered.index.unique():
    avery_price_dict[i] = avery_price_filtered.loc[i, ['id','buyboxtime','price',
       'high', 'low', 'stock', 'merchant','prime','subscribe',
       'pantry', 'addon', 'min', 'max', 'mapviolation', 'sellers',
       'Category_custom', 'Subcategory_custom']]

/Users/yvettepeng/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


### Drop duplicated values and deal with time

In [63]:
# sales rank dictionary
# deal with duplicated values and time
# log_sales_rank and delta_log_sales_rank
for i in avery_sales_rank_dict:
    if type(avery_sales_rank_dict[i]) != pd.core.series.Series:
        avery_sales_rank_dict[i] = avery_sales_rank_dict[i].sort_values('date').reset_index()
        avery_sales_rank_dict[i].drop_duplicates(inplace = True)
        avery_sales_rank_dict[i]['new_date'] = pd.to_datetime(avery_sales_rank_dict[i]['date'])
        avery_sales_rank_dict[i]['log_rank'] = avery_sales_rank_dict[i]['category_rank1'].map(lambda x: math.log(x))
        avery_sales_rank_dict[i]['delta_rank'] = avery_sales_rank_dict[i]['log_rank'].diff()
        a = avery_sales_rank_dict[i].index[0]
        avery_sales_rank_dict[i].drop(a, inplace = True)
        avery_sales_rank_dict[i]['new_date'] = avery_sales_rank_dict[i]['new_date'].dt.strftime('%Y-%m-%d')
        avery_sales_rank_dict[i]['new_date'] = pd.to_datetime(avery_sales_rank_dict[i]['new_date'])

In [64]:
# reset index of sales rank dictionary
for i in avery_sales_rank_dict:
    if type(avery_sales_rank_dict[i]) != pd.core.series.Series:
        avery_sales_rank_dict[i] = avery_sales_rank_dict[i].reset_index(drop = True)

In [65]:
# deal with time of review dictionary
for i in avery_review_dict:
    if type(avery_review_dict[i]) != pd.core.series.Series:
        avery_review_dict[i] = avery_review_dict[i].sort_values('date').reset_index()
        avery_review_dict[i].drop_duplicates(inplace = True)
        avery_review_dict[i]['date'] = pd.to_datetime(avery_review_dict[i]['date'])
        avery_review_dict[i]['date'] = avery_review_dict[i]['date'].dt.strftime('%Y-%m-%d')
        avery_review_dict[i]['date'] = pd.to_datetime(avery_review_dict[i]['date'])

In [66]:
# deal with time of price dictionary
for i in avery_price_dict:
    if type(avery_price_dict[i]) != pd.core.series.Series:
        avery_price_dict[i] = avery_price_dict[i].drop(labels = 'id',axis = 1)
        avery_price_dict[i] = avery_price_dict[i].sort_values('buyboxtime').reset_index()
        avery_price_dict[i].drop_duplicates(inplace = True)
        avery_price_dict[i]['date'] = pd.to_datetime(avery_price_dict[i]['buyboxtime'])
        avery_price_dict[i]['date'] = avery_price_dict[i]['date'].dt.strftime('%Y-%m-%d')
        avery_price_dict[i]['date'] = pd.to_datetime(avery_price_dict[i]['date'])

### Aggregate Review Rating features

First, try one feature

In [67]:
for i in avery_sales_rank_dict:
    if i in avery_review_dict:
        for j in avery_sales_rank_dict[i].index.unique():
            if type(avery_review_dict[i]) != pd.core.series.Series:
                avery_sales_rank_dict[i].loc[j, "stars_recent_10"] = \
                avery_review_dict[i][avery_review_dict[i]['date'] <= \
                                     avery_sales_rank_dict[i].loc[j, 'date']].tail(10)['stars'].mean()
    

In [68]:
for i in avery_sales_rank_dict:
    if i in avery_review_dict:
        for j in avery_sales_rank_dict[i].index.unique():
            if type(avery_review_dict[i]) != pd.core.series.Series:
                avery_sales_rank_dict[i].loc[j, 'stars_recent_oneweek'] = avery_review_dict[i][(avery_review_dict[i]['date'] \
                    <= avery_sales_rank_dict[i].loc[j, 'new_date']) & (avery_review_dict[i]['date'] >= \
                    (avery_sales_rank_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 6)))]['stars'].mean()
                
                avery_sales_rank_dict[i].loc[j, 'stars_recent_onemonth'] = avery_review_dict[i][(avery_review_dict[i]['date'] \
                    <= avery_sales_rank_dict[i].loc[j, 'new_date']) & (avery_review_dict[i]['date'] >= \
                    (avery_sales_rank_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 29)))]['stars'].mean()
                
                avery_sales_rank_dict[i].loc[j, 'stars_avg'] = avery_review_dict[i][avery_review_dict[i]['date'] \
                    <= avery_sales_rank_dict[i].loc[j, 'new_date']]['stars'].mean()
                
                avery_sales_rank_dict[i].loc[j, 'review_acc'] = avery_review_dict[i][avery_review_dict[i]['date'] \
                    <= avery_sales_rank_dict[i].loc[j, 'new_date']]['date'].count()
                
                avery_sales_rank_dict[i].loc[j, 'reviewnum_oneweek'] = avery_review_dict[i][(avery_review_dict[i]['date'] \
                    <= avery_sales_rank_dict[i].loc[j, 'new_date']) & (avery_review_dict[i]['date'] >= \
                    (avery_sales_rank_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 6)))]['date'].count()
                
                
                avery_sales_rank_dict[i].loc[j, 'reviewnum_onemonth'] = avery_review_dict[i][(avery_review_dict[i]['date'] \
                    <= avery_sales_rank_dict[i].loc[j, 'new_date']) & (avery_review_dict[i]['date'] >= \
                    (avery_sales_rank_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 29)))]['date'].count()
                
                avery_sales_rank_dict[i].loc[j, 'author_acc'] = avery_review_dict[i][(avery_review_dict[i]['date'] \
                    <= avery_sales_rank_dict[i].loc[j,'new_date']) & (avery_review_dict[i]['author'].str.contains('Customer') == False) \
                    & (avery_review_dict[i]['author'].str.contains('customer') == False)]['date'].count() / \
                    avery_review_dict[i][avery_review_dict[i]['date'] <= avery_sales_rank_dict[i].loc[j,'new_date']]['date'].count()
                
                avery_sales_rank_dict[i].loc[j, 'num_1_month'] = avery_review_dict[i][(avery_review_dict[i]['date'] \
                    <= avery_sales_rank_dict[i].loc[j, 'date']) & (avery_review_dict[i]['date'] >= \
                    (avery_sales_rank_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 29))) & \
                    (avery_review_dict[i]['stars'] == 1)]['date'].count()
                
                avery_sales_rank_dict[i].loc[j, 'num_5_month'] = avery_review_dict[i][(avery_review_dict[i]['date'] <= \
                    avery_sales_rank_dict[i].loc[j, 'date']) & (avery_review_dict[i]['date'] >= \
                    (avery_sales_rank_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 29))) & \
                    (avery_review_dict[i]['stars'] == 5)]['date'].count()
                
    

In [69]:
avery_sales_rank_dict['B00004Z5QO']

,id,source,date,category_id1,category_name1,category_rank1,category_id2,category_name2,category_rank2,new_date,...,stars_recent_10,stars_recent_oneweek,stars_recent_onemonth,stars_avg,review_acc,reviewnum_oneweek,reviewnum_onemonth,author_acc,num_1_month,num_5_month
0,B00004Z5QO,amazon,2019-03-12 19:49:09,office-products/1069410,Laser & Inkjet Printer Labels,2.0,office-products,Office Products,520.0,2019-03-12,...,4.6,NaN,5.0,4.61236,534.0,0.0,2.0,0.930712,0.0,2.0
1,B00004Z5QO,amazon,2019-03-13 20:08:07,office-products/1069410,Office Products,4.0,office-products,Office Products,528.0,2019-03-13,...,4.6,NaN,5.0,4.61236,534.0,0.0,2.0,0.930712,0.0,2.0
2,B00004Z5QO,amazon,2019-03-14 20:52:36,office-products/1069410,Laser & Inkjet Printer Labels,1.0,office-products,Office Products,417.0,2019-03-14,...,4.6,NaN,5.0,4.61236,534.0,0.0,2.0,0.930712,0.0,2.0
3,B00004Z5QO,amazon,2019-03-15 19:37:16,office-products/1069410,Office Products,2.0,office-products,Office Products,503.0,2019-03-15,...,4.6,NaN,5.0,4.61236,534.0,0.0,2.0,0.930712,0.0,2.0
4,B00004Z5QO,amazon,2019-03-16 19:13:30,office-products/1069410,Laser & Inkjet Printer Labels,3.0,office-products,Office Products,572.0,2019-03-16,...,4.6,NaN,5.0,4.61236,534.0,0.0,2.0,0.930712,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,B00004Z5QO,amazon,2019-11-26 20:03:14,office-products/1069410,Laser & Inkjet Printer Labels,5.0,office-products,Office Products,634.0,2019-11-26,...,4.0,NaN,NaN,4.59823,565.0,0.0,0.0,0.932743,0.0,0.0
260,B00004Z5QO,amazon,2019-11-27 20:24:49,office-products/1069410,Laser & Inkjet Printer Labels,7.0,office-products,Office Products,733.0,2019-11-27,...,4.0,NaN,NaN,4.59823,565.0,0.0,0.0,0.932743,0.0,0.0
261,B00004Z5QO,amazon,2019-11-28 20:17:04,office-products/1069410,Laser & Inkjet Printer Labels,8.0,office-products,Office Products,895.0,2019-11-28,...,4.0,NaN,NaN,4.59823,565.0,0.0,0.0,0.932743,0.0,0.0
262,B00004Z5QO,amazon,2019-11-29 19:09:49,office-products/1069410,Laser & Inkjet Printer Labels,8.0,office-products,Office Products,1545.0,2019-11-29,...,4.0,NaN,NaN,4.59823,565.0,0.0,0.0,0.932743,0.0,0.0


### Aggregate Word Count Features

In [70]:
for i in avery_sales_rank_dict:
    try:
        if i in avery_review_dict:
            for j in avery_sales_rank_dict[i].index.unique():
                if type(avery_review_dict[i]) != pd.core.series.Series:
                    avery_sales_rank_dict[i].loc[j, 'avg_word_count'] = \
                        avery_review_dict[i][avery_review_dict[i]['date'] <= \
                        avery_sales_rank_dict[i].loc[j, 'date']]['text'].apply(lambda x: x.count(' ')+1).mean()
                    
                    avery_sales_rank_dict[i].loc[j, 'avg_word_count_10'] = \
                        avery_review_dict[i][avery_review_dict[i]['date'] <= \
                        avery_sales_rank_dict[i].loc[j, 'date']].tail(10)['text'].apply(lambda x: x.count(' ')+1).mean()        
    except:
        continue                                                                             

### Aggregate Reviewer Status

In [72]:
for i in avery_sales_rank_dict:
    if i in avery_review_dict:
        for j in avery_sales_rank_dict[i].index.unique():
            if type(avery_review_dict[i]) != pd.core.series.Series:
                avery_sales_rank_dict[i].loc[j, 'verified_acc'] = \
                    (avery_review_dict[i][(avery_review_dict[i]['date'] <= avery_sales_rank_dict[i].loc[j, 'date']) & \
                    (avery_review_dict[i]['verified'] == True)]['date'].count()) / \
                    (avery_review_dict[i][avery_review_dict[i]['date'] <= \
                                        avery_sales_rank_dict[i].loc[j, 'date']]['date'].count()) 
            
                avery_sales_rank_dict[i].loc[j, 'verified_acc_oneweek'] = \
                    (avery_review_dict[i][(avery_review_dict[i]['date'] <= avery_sales_rank_dict[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['date'] >= (avery_sales_rank_dict[i].loc[j,'new_date'] - \
                    datetime.timedelta(days = 6))) & (avery_review_dict[i]['verified'] == True)]['date'].count()) / \
                    (avery_review_dict[i][(avery_review_dict[i]['date'] <= avery_sales_rank_dict[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['date'] >= (avery_sales_rank_dict[i].loc[j,'new_date'] - \
                                                      datetime.timedelta(days = 6)))]['date'].count()) 
       

/Users/yvettepeng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in long_scalars


### Aggregate Price Features

In [73]:
avery_price_dict['B00004Z5QO']['sellers'].unique()

array([4])

In [77]:
for i in avery_sales_rank_dict:
    if i in avery_price_dict:
        for j in range(4,len(avery_sales_rank_dict[i].index)):
            if type(avery_price_dict[i]) != pd.core.series.Series:
                if any(avery_price_dict[i]['buyboxtime'] <= avery_sales_rank_dict[i].loc[j, 'date']):
                        avery_sales_rank_dict[i].loc[j, 'latest_price'] = avery_price_dict[i][avery_price_dict[i]['buyboxtime'] <= \
                        avery_sales_rank_dict[i].loc[j, 'date']]['price'].iloc[-1] 
                else:
                    avery_sales_rank_dict[i].loc[j, 'latest_price'] = np.nan

In [78]:
for i in avery_sales_rank_dict:
        if i in avery_price_dict:
            for j in range(4,len(avery_sales_rank_dict[i].index)):
                if type(avery_price_dict[i]) != pd.core.series.Series:
                    if any(avery_price_dict[i]['date'] == avery_sales_rank_dict[i].loc[j, 'new_date']):
                        avery_sales_rank_dict[i].loc[j, 'first_price'] = \
                        avery_price_dict[i][avery_price_dict[i]['date'] == \
                        avery_sales_rank_dict[i].loc[j, 'new_date']]['price'].iloc[-1]
                    else:
                        avery_sales_rank_dict[i].loc[j, 'first_price'] = np.nan

In [79]:
# fill NAs with the mean price
for i in avery_sales_rank_dict:
    avery_sales_rank_dict[i]['latest_price'] = avery_sales_rank_dict[i]['latest_price'].fillna(avery_sales_rank_dict[i]['latest_price'].mean())
    avery_sales_rank_dict[i]['first_price'] = avery_sales_rank_dict[i]['first_price'].fillna(avery_sales_rank_dict[i]['first_price'].mean())

In [80]:
avery_sales_rank_dict['B00004Z5QO']

,id,source,date,category_id1,category_name1,category_rank1,category_id2,category_name2,category_rank2,new_date,...,reviewnum_onemonth,author_acc,num_1_month,num_5_month,avg_word_count,avg_word_count_10,verified_acc,verified_acc_oneweek,latest_price,first_price
0,B00004Z5QO,amazon,2019-03-12 19:49:09,office-products/1069410,Laser & Inkjet Printer Labels,2.0,office-products,Office Products,520.0,2019-03-12,...,2.0,0.930712,0.0,2.0,23.818352,16.2,0.878277,NaN,8.679238,8.954048
1,B00004Z5QO,amazon,2019-03-13 20:08:07,office-products/1069410,Office Products,4.0,office-products,Office Products,528.0,2019-03-13,...,2.0,0.930712,0.0,2.0,23.818352,16.2,0.878277,NaN,8.679238,8.954048
2,B00004Z5QO,amazon,2019-03-14 20:52:36,office-products/1069410,Laser & Inkjet Printer Labels,1.0,office-products,Office Products,417.0,2019-03-14,...,2.0,0.930712,0.0,2.0,23.818352,16.2,0.878277,NaN,8.679238,8.954048
3,B00004Z5QO,amazon,2019-03-15 19:37:16,office-products/1069410,Office Products,2.0,office-products,Office Products,503.0,2019-03-15,...,2.0,0.930712,0.0,2.0,23.818352,16.2,0.878277,NaN,8.679238,8.954048
4,B00004Z5QO,amazon,2019-03-16 19:13:30,office-products/1069410,Laser & Inkjet Printer Labels,3.0,office-products,Office Products,572.0,2019-03-16,...,2.0,0.930712,0.0,2.0,23.818352,16.2,0.878277,NaN,8.679238,8.954048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,B00004Z5QO,amazon,2019-11-26 20:03:14,office-products/1069410,Laser & Inkjet Printer Labels,5.0,office-products,Office Products,634.0,2019-11-26,...,0.0,0.932743,0.0,0.0,23.961062,28.6,0.883186,NaN,7.580000,8.954048
260,B00004Z5QO,amazon,2019-11-27 20:24:49,office-products/1069410,Laser & Inkjet Printer Labels,7.0,office-products,Office Products,733.0,2019-11-27,...,0.0,0.932743,0.0,0.0,23.961062,28.6,0.883186,NaN,7.580000,8.954048
261,B00004Z5QO,amazon,2019-11-28 20:17:04,office-products/1069410,Laser & Inkjet Printer Labels,8.0,office-products,Office Products,895.0,2019-11-28,...,0.0,0.932743,0.0,0.0,23.961062,28.6,0.883186,NaN,7.580000,8.954048
262,B00004Z5QO,amazon,2019-11-29 19:09:49,office-products/1069410,Laser & Inkjet Printer Labels,8.0,office-products,Office Products,1545.0,2019-11-29,...,0.0,0.932743,0.0,0.0,23.961062,28.6,0.883186,NaN,7.580000,8.954048


### Remove problematic products with only 1 records

In [81]:
problem_products=[]
for i in avery_sales_rank_dict:
    if type(avery_sales_rank_dict[i]['category_rank1'])!= pd.core.series.Series:
        problem_products.append(i)
problem_products

[]

In [82]:
for item in problem_products:
    del avery_sales_rank_dict[item]

In [100]:
new_dict_avery_sales_rank = dict((i,avery_sales_rank_dict[k]) for i,k in enumerate(sorted(avery_sales_rank_dict.keys())))

In [136]:
keys = avery_sales_rank_dict.keys()

In [123]:
test1 = avery_sales_rank_dict['B00004Z5QO']
test2 = avery_sales_rank_dict['B00004Z5SB']

In [124]:
test3 = pd.concat([test1, test2], ignore_index=True)

In [132]:
test3.head()

,id,source,date,category_id1,category_name1,category_rank1,category_id2,category_name2,category_rank2,new_date,...,reviewnum_onemonth,author_acc,num_1_month,num_5_month,avg_word_count,avg_word_count_10,verified_acc,verified_acc_oneweek,latest_price,first_price
0,B00004Z5QO,amazon,2019-03-12 19:49:09,office-products/1069410,Laser & Inkjet Printer Labels,2.0,office-products,Office Products,520.0,2019-03-12,...,2.0,0.930712,0.0,2.0,23.818352,16.2,0.878277,NaN,8.679238,8.954048
1,B00004Z5QO,amazon,2019-03-13 20:08:07,office-products/1069410,Office Products,4.0,office-products,Office Products,528.0,2019-03-13,...,2.0,0.930712,0.0,2.0,23.818352,16.2,0.878277,NaN,8.679238,8.954048
2,B00004Z5QO,amazon,2019-03-14 20:52:36,office-products/1069410,Laser & Inkjet Printer Labels,1.0,office-products,Office Products,417.0,2019-03-14,...,2.0,0.930712,0.0,2.0,23.818352,16.2,0.878277,NaN,8.679238,8.954048
3,B00004Z5QO,amazon,2019-03-15 19:37:16,office-products/1069410,Office Products,2.0,office-products,Office Products,503.0,2019-03-15,...,2.0,0.930712,0.0,2.0,23.818352,16.2,0.878277,NaN,8.679238,8.954048
4,B00004Z5QO,amazon,2019-03-16 19:13:30,office-products/1069410,Laser & Inkjet Printer Labels,3.0,office-products,Office Products,572.0,2019-03-16,...,2.0,0.930712,0.0,2.0,23.818352,16.2,0.878277,NaN,8.679238,8.954048


In [127]:
len(test3)

528

In [128]:
len(test1)

264

In [147]:
reg_table = pd.DataFrame()
# pull out interested products to form a new dataframe
for i in avery_sales_rank_dict:
    df_bin = avery_sales_rank_dict[i]
    reg_table = pd.concat([reg_table, df_bin],ignore_index=True)

/Users/yvettepeng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [149]:
# The first product is what we don't want, (not in common), but in the above loop, it's more convenient to include that, so we delete it now.
# final_df = a[a['id'] != 'B00000JFNV']

# put our interested columns in the final dataframe.
reg_table = reg_table.reset_index(drop = True)[['id', 'date', 'category_name1',
       'category_rank1','log_rank', 'delta_rank',
       'stars_recent_10', 'stars_recent_oneweek',
       'stars_recent_onemonth', 'stars_avg', 'review_acc', 'reviewnum_oneweek',
       'reviewnum_onemonth', 'avg_word_count', 'verified_acc',
       'verified_acc_oneweek','avg_word_count_10','num_1_month', 'num_5_month',
        'latest_price','first_price','author_acc']]


# replace the NAs with 0
reg_table = reg_table.fillna(0)
reg_table.drop_duplicates(inplace = True)
reg_table.to_csv('Avery_Regression_Table.csv')

In [150]:
reg_table.columns

Index(['id', 'date', 'category_name1', 'category_rank1', 'log_rank',
       'delta_rank', 'stars_recent_10', 'stars_recent_oneweek',
       'stars_recent_onemonth', 'stars_avg', 'review_acc', 'reviewnum_oneweek',
       'reviewnum_onemonth', 'avg_word_count', 'verified_acc',
       'verified_acc_oneweek', 'avg_word_count_10', 'num_1_month',
       'num_5_month', 'latest_price', 'first_price', 'author_acc'],
      dtype='object')